In [1]:
#李子霄 2017301020223 第三次作业

In [35]:
import time
import math
import numpy as np
import time
import sympy
import numba
from numba import jit
from sympy import *
import matplotlib as mpl
import matplotlib.pyplot as plt

作业一

In [32]:
start_time=time.time()
N=10000
s1 = np.arange(1,N,dtype=np.float32) #产⽣[1,...,99]的单精度浮点数队列
s_up = 1 / s1
s_up_sum = s_up.sum()

s2 = np.arange(N-1,0,-1,dtype=np.float32) #产⽣[99,...,1]的单精度浮点数队列
s_down = 1 / s2
s_down_sum = s_down.sum()
print(s_up_sum)
print(s_down_sum)
end_time=time.time()
total_time=end_time-start_time
total_time

9.787508
9.787506


0.0020008087158203125

用numpy改写的计算花费时间为0.002000

In [31]:
start_time_2=time.time()
N=10000
s_up=float(0)
for n in range(1,N):#这表示从1到9999
    s_up+=float(1/n)
s_down=float(0)
for n in range(N-1,0,-1):#这表示从9999到1
    s_down+=float(1/n)
print(s_up)
print(s_down)
end_time_2=time.time()
total_time_2=end_time_2-start_time_2
total_time_2

9.787506036044348
9.787506036044386


0.007010221481323242

不用numpy而是使用自带的float型的话用时0.007010,比numpy慢三倍；我认为

np.arange(1,N,dtype=np.float32)

这个语句还是拖慢了计算速度，接下来删掉这一句再来比较用时

In [33]:
start_time=time.time()
N=10000
s1 = np.arange(1,N) #产⽣[1,...,99]的单精度浮点数队列
s_up = 1 / s1
s_up_sum = s_up.sum()

s2 = np.arange(N-1,0,-1) #产⽣[99,...,1]的单精度浮点数队列
s_down = 1 / s2
s_down_sum = s_down.sum()
print(s_up_sum)
print(s_down_sum)
end_time=time.time()
total_time=end_time-start_time
total_time

9.787506036044384
9.787506036044382


0.0010004043579101562

果然，删掉

np.arange(N-1,0,-1,dtype=np.float32)

中的

dtype=np.float32

还能加快一倍速度；但是这其实有点违背计算的本意，把s_up单独的每项不按浮点数计，最好不要删掉...

这说明有比for语句更优的遍历方法

作业二

In [167]:
@jit(nopython=True)
def getnext_sin(x,a,n):
    nexta=numba.float32(-x**2/((2*n-1)*(2*n-2))*a)
    return nexta
#sin的递推公式

In [168]:
N=10000
@jit(nopython=True)
def summation_sin(x):
    a=x
    sers=x
    for n in range(2,N+1):
        sers+=np.float32(getnext_sin(x,a,n))
        a=np.float32(getnext_sin(x,a,n))
    if np.absolute(sers-np.sin(x))<10**-5:
        gx=x
        cor=sers
    return sers
#sin的泰勒展开

In [190]:
@jit(nopython=True)
def getnext_cos(x,a,n):
    nexta=numba.float32(-x**2/((2*n-1)*(2*n-2))*a)
    return nexta
#cos的递推公式

In [191]:
N=10000
@jit(nopython=True)
def summation_cos(x):
    a=1
    sers=1
    for n in range(2,N+1):
        sers+=np.float32(getnext_cos(x,a,n))
        a=np.float32(getnext_cos(x,a,n))
    if np.absolute(sers-np.sin(x))<10**-5:
        gx=x
        cor=sers
    return sers
#cos的泰勒展开

In [195]:
#定义主函数，以任意角度值为输入
def main(x):
  if x>=0:
    if float(x-int(x/2/np.pi)*2*pi)>=0 and float(x-int(x/2/np.pi)*2*pi)<=float(np.pi/2):
        print(summation_sin(x))
    if float(x-int(x/2/np.pi)*2*pi)>=float(np.pi/2) and float(x-int(x/2/np.pi)*2*pi)<=float(np.pi):
        print(summation_cos(x-np.pi/2))
    if float(x-int(x/2/np.pi)*2*pi)>=float(np.pi) and float(x-int(x/2/np.pi)*2*pi)<=float(3*np.pi/2):
        print(-summation_sin(x-np.pi))
    if float(x-int(x/2/np.pi)*2*pi)>=float(3*np.pi/2) and float(x-int(x/2/np.pi)*2*pi)<=float(2*np.pi):
        print(-summation_cos(x-3/2*np.pi))
  if x<0:#如果输入的弧度值小于0，取其绝对值
    x=np.absolute(x)
    if float(x-int(x/2/np.pi)*2*pi)>=0 and float(x-int(x/2/np.pi)*2*pi)<=float(np.pi/2):
        print(-summation_sin(x))
    if float(x-int(x/2/np.pi)*2*pi)>=float(np.pi/2) and float(x-int(x/2/np.pi)*2*pi)<=float(np.pi):
        print(-summation_cos(x-np.pi/2))
    if float(x-int(x/2/np.pi)*2*pi)>=float(np.pi) and float(x-int(x/2/np.pi)*2*pi)<=float(3*np.pi/2):
        print(summation_sin(x-np.pi))
    if float(x-int(x/2/np.pi)*2*pi)>=float(3*np.pi/2) and float(x-int(x/2/np.pi)*2*pi)<=float(2*np.pi):
        print(summation_cos(x-3/2*np.pi))

思路：

（1）先区分输入值为正还是为负，再把任何一个输入的弧度值加减2*pi的整数倍，移动到0至2*pi范围内，即先取弧度绝对值，再减去2*pi的整数倍；

（2）在（0,pi/2）范围内用sin(x)计算

（3）在（pi/2,pi）范围内用cos（x-pi/2）计算

（4）在(pi,3/2*pi)范围内用-sin(x-pi)计算

（5）在（3，2*pi）范围内用-cos(x-3/2*pi)计算

In [196]:
main(-1)

-0.8414709802583705
